In [103]:
from metar import Metar
from datetime import datetime, timedelta
from metar import Metar
from util.utils import *
import pandas as pd
import re


In [104]:

def interpretaMetaR(metarString):

    if("METAF" in metarString):
        metarString = metarString.replace('METAF', 'METAR')
        metarString = metarString.replace('\n', '')


    station = None
    type = None
    hora_formated = None
    temperature = None
    dew_point = None
    windDirection = None
    windVelocity = None
    visibility = None
    pressure = None
    weather = None
    sky = None
    skyFeet= None

    try: 

        obs = Metar.Metar(metarString[:-1])

        for item in obs.string().split('\n'):
            if("station" in item):
                station = item.split(" ")[1]
            if("type" in item):
                type = " ".join(item.split(" ")[1:])
            if("time" in item):
                timeZ = " ".join(item.split(" ")[1:])
                data_obj = datetime.strptime(timeZ, "%a %b %d %H:%M:%S %Y")
                nova_data_obj = data_obj - timedelta(hours=3)
                hora_formated = nova_data_obj.strftime("%a %b %d %H:%M:%S %Y")
            if("temperature" in item):
                temperature = item.split(" ")[1]
            if("dew point" in item):
                dew_point = item.split(" ")[2]
            if("wind" in item):
                wind = (item.split(" ")[1:])
                windDirection = wind[0]
                windVelocity = wind[2]
            if("visibility" in item):
                visibility = item.split(" ")[1]
            if("pressure" in item):
                pressure = item.split(" ")[1]
            if("weather" in item):
                weather = " ".join(item.split(" ")[1:])
            if("sky" in item):
                skyString = " ".join(item.split(" ")[1:])
                if(' at ' in skyString):
                    sky = ''.join(skyString.split(' at ')[0])
                else:
                    sky = ''.join(skyString.split(',')[0])
                
                allFeets = re.findall(r"(\d+)",skyString)
                if len(allFeets) > 0:
                    skyFeet = allFeets[0]
    except:
        pass

    return station, type, hora_formated, temperature,dew_point,windDirection,windVelocity,visibility,pressure,weather,sky,skyFeet

def direction_to_degrees(df):
    direction_degrees = {
    'N': 0,
    'NNE': 22.5,
    'NE': 45,
    'ENE': 67.5,
    'E': 90,
    'ESE': 112.5,
    'SE': 135,
    'SSE': 157.5,
    'S': 180,
    'SSW': 202.5,
    'SW': 225,
    'WSW': 247.5,
    'W': 270,
    'WNW': 292.5,
    'NW': 315,
    'NNW': 337.5,
    'calm': 0,
    'variable': 0,
    }

    df['windDirection'] = df['windDirection'].map(direction_degrees)
    return df

def defineIndex(df):
    df = df.sort_values(by='hora_formatted')
    df.set_index('hora_formatted', inplace=True)
    df.index = pd.to_datetime(df.index, format='%A, %B %d, %Y %H:%M:%S')
    return df

def to_dummies(df):
    df_Dummies = pd.get_dummies(df['sky'])
    df_final = pd.concat([df, df_Dummies], axis=1)

    return df_final

def sort_df(df):
    df = df.drop_duplicates()
    df = df.dropna()
    df = df.sort_values(by=['aero','hora_formatted'])

    return df
def calculate_average(series):
    result = []
    for i in range(len(series) - 1):
        current_value = series.iloc[i]
        next_value = series.iloc[i + 1]
        average = (current_value + next_value) / 2
        result.append(average)
    return pd.Series(result, index=series.index[:-1])

def pipemediaTempoMetar(df, colunas):
    listSeries = []
    non_numeric_columns = []

    for coluna in colunas:
        if pd.api.types.is_numeric_dtype(df[coluna]):
            df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
            df_resampled_temp = df[coluna].resample('30T').mean()
            df_resampled_temp_filled = df_resampled_temp.fillna(method='ffill')
            tempSeries = calculate_average(df_resampled_temp_filled)
            listSeries.append(tempSeries)
        else:
            non_numeric_columns.append(coluna)

    dataframe_resultante = pd.concat(listSeries, axis=1)

    # Criar um novo DataFrame para armazenar as colunas não numéricas
    df_nao_numericas = df[non_numeric_columns]

    # Redefinir os índices dos DataFrames antes da concatenação
    dataframe_resultante.reset_index(drop=True, inplace=True)
    df_nao_numericas.reset_index(drop=True, inplace=True)

    # Concatenar o DataFrame resultante com o DataFrame das colunas não numéricas
    resultado_final = pd.concat([dataframe_resultante, df_nao_numericas], axis=1)

    return resultado_final

def read_metar(tabela):
    if (tabela == "metar"):
        df = openDataframe("metar_bronze.csv")
        resultado_df =  df['metar'].apply(interpretaMetaR)
    elif (tabela == "metaf"):
        df = openDataframe("metaf_bronze.csv")
        resultado_df =  df['metaf'].apply(interpretaMetaR)
    
    resultado_df =  pd.DataFrame(resultado_df.tolist(),columns=['station', 'type', 'metar_hora_formated', 'station','dew_point','windDirection','windVelocity','visibility','pressure','weather','sky','skyFeet'] )
    df_bronze = pd.concat([df, resultado_df], axis=1)
    df_bronze = direction_to_degrees(df_bronze)
    df_bronze = sort_df(df_bronze)
    df_bronze = defineIndex(df_bronze)

    df_media = pipemediaTempoMetar(df_bronze,['hora','aero','dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'])
    df_no_columns = df_bronze.drop(['dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'],axis=1)
    return df_media,df_bronze



In [105]:
df,df2 = read_metar("metar")

df.col

AttributeError: 'Series' object has no attribute 'columns'